In [1]:
# import monai
import pydicom
import pathlib
from glob import glob
import matplotlib.pyplot as plt

# Connecting to XNAT's REST API 

In [2]:
import pyxnat
mode = 'local_machine'
server_dict = dict(local_machine='http://localhost:80' , inside_xnat='http://xnat-web:8080')
interface = pyxnat.Interface(server=server_dict[mode], user='admin', password='admin')

# Level 1. Project

In [3]:
print('projects:', interface.select.projects().get())

projects: ['project', 'demo1']


In [4]:
project_ID = 'demo1'
project = interface.select.project(project_ID)

### List of users in the project

In [ ]:
print('users:', project.users())

### Resources in the project

In [ ]:
print('resources:', project.resources().get())

### List of Subjects in the project

In [ ]:
print('subjects:', project.subjects().get())

# Level 2. Subject

In [5]:
subject_ID = 'XNAT_S00004'
subject = project.subject(subject_ID)

subject
# if not subject.exists():
#     print('subject does not exist')
#     subject.insert()

<Subject Object> XNAT_S00004 `10000019` (project: demo1) (Gender: U) 1 experiment http://localhost:80/data/projects/demo1/subjects/XNAT_S00004?format=html

# Level 3. Experiments

### List of experiments

In [ ]:
project.experiments().get()

In [ ]:
print('experiments:',subject.experiments().get())

### Setting the experiment

In [ ]:
experiment_ID = 'XNAT_E00004'
experiment = subject.experiment(experiment_ID)
experiment

### Creating a new experiment

In [ ]:
# experiment.create(experiments='xnat:ctSessionData')

# experiment.attrs.mset({
#     'xnat:mrSessionData/coil' : 'head',
#     'xnat:mrSessionData/marker' : 'right',
# })

# Level 4. Scans

### List of scans in the experiment

In [ ]:
print('scans:', experiment.scans().get() )

In [ ]:
scan_ID = '2'
scan = experiment.scans(scan_ID)
scan

In [ ]:
# List all resources for the scan
for resource in scan.resources().fetchall():
    print(resource)
    # print(resource.id(), resource.label(), resource.datatype())

scan.resource('9').get()

In [ ]:
dicom_resource = scan.resource('DICOM')

dicom_resource.

### Creating a New Scan

In [ ]:
# scan.create(scans='xnat:ctScanData')

# # Setting scan attributes
# scan.attrs.mset({
#     'xnat:ctScanData/parameters/imageType' : 'dicom',
#     'xnat:ctScanData/series_description': 'something'
# })

# Adding DICOM files to a scan
# dicom_resource = scan.resource('DICOM')
# dicom_resource.put_dir()

# Level 5. Resources

In [ ]:
print('scans:', scan.resources().get())

In [ ]:
resource_ID = '9'
resource = scan.resource(resource_ID)
resource.get()

In [ ]:
resource.get()

In [7]:
if mode == 'local_machine':
    path = pathlib.Path(f'xnat-data/archive/{project_ID}/arc001/*/SCANS/*/DICOM/').absolute()
else:
    path = pathlib.Path(f'/data/projects/{project_ID}/experiments/{experiment_ID}/SCANS/{scan_ID}/DICOM/').absolute()

# path
# path.glob('*')
files = list(path.glob('*.dcm'))
files

[]

## Import the DICOM data

In [ ]:
interface.select('/archive').resource(dicom_path).import_(dicom_path, overwrite='append', trigger_pipelines=None)

# Visualization

## Getting the list of dicom images

In [ ]:
mode = 'local'

In [ ]:
project_ID = 'project'
experiment_ID = 'LUNG1-001'
scan_ID = '0'

if mode == 'local':
    path = pathlib.Path(f'xnat-data/archive/{project_ID}/arc001/{experiment_ID}/SCANS/{scan_ID}/DICOM/').absolute()
else:
    path = pathlib.Path(f'/data/projects/{project_ID}/experiments/{experiment_ID}/SCANS/{scan_ID}/DICOM/').absolute()

files = list(path.glob('*.dcm'))
# files

## Loading and visualizing the dicom image

In [ ]:
dcm_file = pydicom.dcmread(files[10])

# Visualize the dicom image
fig = plt.figure()
plt.imshow(dcm_file.pixel_array, cmap=plt.cm.bone)

# plt.savefig('im.jpg')

## Applying transformations using monai library

In [ ]:
im = dcm_file.pixel_array 

from monai.transforms import AdjustContrast
adjuster = AdjustContrast(.2)
im2 = adjuster(im)
plt.imshow(im2, cmap=plt.cm.bone)